In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0217],
        [-0.0396]], grad_fn=<AddmmBackward>)

# Parameter Access

In [3]:
print(net[2].state_dict()) # the second layer

OrderedDict([('weight', tensor([[-0.0286, -0.0025, -0.0926, -0.2612,  0.2987,  0.2849, -0.3087,  0.2815]])), ('bias', tensor([0.0624]))])


## Targeted Parameters

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0624], requires_grad=True)
tensor([0.0624])


In [5]:
net[2].weight.grad == None

True

## All Parameters Once

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([0.0624])

## Collecting Parameters from Nested Blocks

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here.
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1303],
        [-0.1304]], grad_fn=<AddmmBackward>)

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): Linear(in_features=8, out_features=4, bias=True)
      (2): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): Linear(in_features=8, out_features=4, bias=True)
      (2): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): Linear(in_features=8, out_features=4, bias=True)
      (2): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): Linear(in_features=8, out_features=4, bias=True)
      (2): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [10]:
rgnet[0][1][0].bias.data

tensor([ 0.1912, -0.2081,  0.3924,  0.0034,  0.2427, -0.4052, -0.2548, -0.2265])

# Parameter Initialization

## Built-in Initialization

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
    
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0068, 0.0092, 0.0075, 0.0080]), tensor(0.))

In [12]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [13]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4922, -0.4661, -0.6326, -0.3501])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## Custom Initialization

In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print('Init', 
             *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, 8.2343, 0.0000],
        [-0.0000, 6.4215, 7.0735, 0.0000]], grad_fn=<SliceBackward>)

In [15]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  9.2343,  1.0000])

# Tired Parameters

In [16]:
# We need to give the shared layer a name so that we can refer to
# its parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(),
                   shared, nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[-0.2359],
        [-0.2463]], grad_fn=<AddmmBackward>)

In [19]:
# Check the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [20]:
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object
# ranther than just having the same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
